In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/SCP1/Dataset"

train_dir = f"{DATA_DIR}/train"
val_dir   = f"{DATA_DIR}/val"
test_dir  = f"{DATA_DIR}/test"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-image opencv-python tensorflow matplotlib

In [ ]:
import cv2
import numpy as np
from skimage.segmentation import slic
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.measure import shannon_entropy

In [ ]:
def extract_ROI_superpixel(img, num_segments=200, top_k=40):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = rgb2gray(img_rgb)

    segments = slic(img_rgb, n_segments=num_segments, compactness=10)
    scores = []

    for seg_id in np.unique(segments):
        mask = segments == seg_id
        region = gray[mask]

        entropy = shannon_entropy(region)
        contrast = region.std()
        saliency = sobel(gray)[mask].mean()

        score = 0.5*entropy + 0.3*contrast + 0.2*saliency
        scores.append((seg_id, score))

    top_segments = [x[0] for x in sorted(scores, key=lambda x:x[1], reverse=True)[:top_k]]

    masked = np.zeros_like(img_rgb)
    for seg_id in top_segments:
        masked[segments == seg_id] = img_rgb[segments == seg_id]

    return masked

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

IMG_SIZE = (224,224)
def load_roi(path):
    path = path.numpy().decode("utf-8")
    img = cv2.imread(path)
    img = cv2.resize(img, IMG_SIZE)
    img = extract_ROI_superpixel(img, 200, 50)
    return (img/255.).astype(np.float32)
def load_normal(path):
    path = path.numpy().decode("utf-8")
    img = cv2.imread(path)
    img = cv2.resize(img, IMG_SIZE)
    return (img/255.).astype(np.float32)



def tf_loader_roi(path, label):
    img = tf.py_function(load_roi, [path], tf.float32)
    img.set_shape((*IMG_SIZE,3))
    return img, tf.cast(label, tf.float32)
def tf_loader_vanilla(path, label):
    img = tf.py_function(load_normal, [path], tf.float32)
    img.set_shape((*IMG_SIZE,3))
    return img, tf.cast(label, tf.float32)


def build_dataset(root, vanilla=False):
    paths,labels = [],[]

    for cls in ["NORMAL","PNEUMONIA"]:
        label = 0 if cls=="NORMAL" else 1
        folder = f"{root}/{cls}"

        for f in os.listdir(folder):
            paths.append(f"{folder}/{f}")
            labels.append(label)

    ds = tf.data.Dataset.from_tensor_slices((paths,labels))

    if vanilla:
        ds = ds.map(tf_loader_vanilla, num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(tf_loader_roi, num_parallel_calls=tf.data.AUTOTUNE)

    return ds.shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE)


train_roi = build_dataset(train_dir, vanilla=False)
val_roi   = build_dataset(val_dir, vanilla=False)
test_roi  = build_dataset(test_dir, vanilla=False)

train_van = build_dataset(train_dir, vanilla=True)
val_van   = build_dataset(val_dir, vanilla=True)
test_van  = build_dataset(test_dir, vanilla=True)


In [ ]:
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        layers.Input(shape=(224,224,3)),
        layers.Conv2D(32,3,activation='relu'), layers.MaxPool2D(),
        layers.Conv2D(64,3,activation='relu'), layers.MaxPool2D(),
        layers.Conv2D(128,3,activation='relu'), layers.MaxPool2D(),
        layers.Flatten(),
        layers.Dense(128,activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1,activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','AUC'])
    return model


In [ ]:
roi_model = build_model()
history_roi = roi_model.fit(train_roi, validation_data=val_roi, epochs=10)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 241s 358ms/step - AUC: 0.7871 - accuracy: 0.7481 - loss: 0.6586 - val_AUC: 0.6641 - val_accuracy: 0.6250 - val_loss: 0.7779
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 242s 232ms/step - AUC: 0.9756 - accuracy: 0.9286 - loss: 0.2159 - val_AUC: 0.7109 - val_accuracy: 0.7500 - val_loss: 0.9089
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 232s 273ms/step - AUC: 0.9947 - accuracy: 0.9506 - loss: 0.1076 - val_AUC: 0.7578 - val_accuracy: 0.6875 - val_loss: 1.0792
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 225s 272ms/step - AUC: 0.9994 - accuracy: 0.9863 - loss: 0.0430 - val_AUC: 0.7578 - val_accuracy: 0.6875 - val_loss: 1.2139
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 260s 258ms/step - AUC: 0.9998 - accuracy: 0.9921 - loss: 0.0237 - val_AUC: 0.7422 - val_accuracy: 0.6875 - val_loss: 1.4952
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 231s 363ms/step - AUC: 1.0000 - accuracy: 1.0000 - loss: 0.0062 - val_AUC: 0.7344 - val_accuracy: 0.6875 - val_loss: 1.9643
Epoch 7/10
32/32 ━━━━━━━━━━━

In [ ]:
van_model = build_model()
history_van = van_model.fit(train_van, validation_data=val_van, epochs=10)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 15s 126ms/step - AUC: 0.6129 - accuracy: 0.5844 - loss: 0.8442 - val_AUC: 0.7188 - val_accuracy: 0.8125 - val_loss: 0.7772
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - AUC: 0.9419 - accuracy: 0.8850 - loss: 0.3147 - val_AUC: 0.8281 - val_accuracy: 0.7500 - val_loss: 0.6793
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - AUC: 0.9819 - accuracy: 0.9335 - loss: 0.1712 - val_AUC: 0.8203 - val_accuracy: 0.7500 - val_loss: 1.0026
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - AUC: 0.9938 - accuracy: 0.9637 - loss: 0.1019 - val_AUC: 0.7812 - val_accuracy: 0.8125 - val_loss: 1.1951
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - AUC: 0.9837 - accuracy: 0.9486 - loss: 0.1482 - val_AUC: 0.8594 - val_accuracy: 0.8125 - val_loss: 0.5839
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - AUC: 0.9993 - accuracy: 0.9965 - loss: 0.0430 - val_AUC: 0.8750 - val_accuracy: 0.6875 - val_loss: 0.7683
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 

In [ ]:
# SAVE MODELS
vanilla_path = "/content/vanilla_cnn.h5"
roi_path     = "/content/roi_cnn.h5"

van_model.save(vanilla_path)
roi_model.save(roi_path)

print("Models saved successfully!")


Models saved successfully!
